In [3]:
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Masking
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold
from tensorflow.keras.optimizers import Adam
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error



In [4]:
conn = sqlite3.connect(r'C:\Users\opper\Master-Arbeit\data\preprocessed\pv_forecast.db')

query = f"""
SELECT "Date time", Location, shortwave_radiation, Temperature, AOI, "Cloud Cover", "Relative Humidity", "kW/kWp"
FROM pv_weather_data
ORDER BY Location, "Date time" ASC
"""
# Daten abrufen und in ein DataFrame laden
df = pd.read_sql(query, conn)
conn.close()

# Ausgabe des DataFrames zum Überprüfen


print("Anzahl NaN-Werte pro Spalte:\n", df.isna().sum())

# Falls es NaN-Werte gibt, betroffene Zeilen anzeigen
if df.isna().sum().sum() > 0:
    print("\nZeilen mit NaN-Werten:")
    print(df[df.isna().any(axis=1)])

Anzahl NaN-Werte pro Spalte:
 Date time              0
Location               0
shortwave_radiation    0
Temperature            0
AOI                    0
Cloud Cover            0
Relative Humidity      0
kW/kWp                 0
dtype: int64


In [5]:
def get_sliding_window(df, datetime_col, feature_cols, target_col, window_size=480, forecast_horizon=96, step_size=1):
    df[datetime_col] = pd.to_datetime(df[datetime_col])
    df = df.sort_values(by=datetime_col).reset_index(drop=True)

    feature_data = df[feature_cols].values
    target_data = df[target_col].values

    X, y = [], []

    for i in range(0, len(df) - window_size - forecast_horizon, step_size):
        X_seq = feature_data[i : i + window_size].copy()

        X_seq[-forecast_horizon:, -1] = -1

        y_seq = target_data[i + window_size : i + window_size + forecast_horizon]

        X.append(X_seq)  # Input-Sequenz
        y.append(y_seq)  # Zielwerte
        
    return np.array(X), np.array(y)




In [6]:
required_columns = ["Date time", "Location", "shortwave_radiation", "Temperature", "AOI", "Cloud Cover", "Relative Humidity", "kW/kWp"]
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise ValueError(f"Fehlende Spalten in der Datenbank: {missing_cols}")

df = df.sort_values(by=["Location", "Date time"]).reset_index(drop=True)

In [8]:
feature_cols = ["shortwave_radiation", "Temperature", "AOI", "Cloud Cover", "Relative Humidity", "kW/kWp"]
target_col = "kW/kWp"

feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

df[feature_cols] = feature_scaler.fit_transform(df[feature_cols])
df[target_col] = target_scaler.fit_transform(df[[target_col]])


X_list, y_list = [], []
location_ids = []


for location in df["Location"].unique():
    print(f"Erstelle Sliding Windows für {location}...")

    df_location = df[df["Location"] == location].copy()

    X, y = get_sliding_window(df_location, datetime_col="Date time", feature_cols=feature_cols, target_col=target_col)

    X_list.append(X)
    y_list.append(y)
    

    location_ids.extend([location] * len(X))

X_final = np.concatenate(X_list, axis=0)  # Alle Input-Sequenzen
y_final = np.concatenate(y_list, axis=0)  # Alle Zielwerte

location_ids = np.array(location_ids)

np.save("X_tensor.npy", X_final)
np.save("y_tensor.npy", y_final)
np.save("location_ids.npy", location_ids)

print(f"Finaler Tensor X Shape: {X_final.shape}")  # (Samples, Timesteps=672, Features=3)
print(f"Finaler Tensor y Shape: {y_final.shape}")  # (Samples, Forecast_Horizon=96)


Erstelle Sliding Windows für bielefeldmel...
Erstelle Sliding Windows für bielefeldref...
Erstelle Sliding Windows für gaithersburg...
Erstelle Sliding Windows für hongkong...
Erstelle Sliding Windows für istanbul...
Erstelle Sliding Windows für victoria14...
Erstelle Sliding Windows für victoria15...
Erstelle Sliding Windows für victoria16...
Erstelle Sliding Windows für victoria17...
Finaler Tensor X Shape: (438007, 480, 6)
Finaler Tensor y Shape: (438007, 96)


In [9]:
print(f"Finaler Tensor y Shape: {location_ids.shape}")  # (Samples, Forecast_Horizon=96)

Finaler Tensor y Shape: (438007,)


In [28]:
# Lade die gespeicherten Tensoren
X_final = np.load("X_tensor.npy")
y_final = np.load("y_tensor.npy")
location_ids = np.load("location_ids.npy")  # Lade die Standort-IDs

# Wähle einen Standort aus, z. B. "Bielefeld"
location = "istanbul"

# Finde die Indizes, an denen dieser Standort vorkommt
indices = np.where(location_ids == location)[0]

# Filtere die Sliding Windows für diesen Standort
X_location = X_final[indices]
y_location = y_final[indices]

print(f"Sliding Windows für {location}: {X_location.shape}")


Sliding Windows für istanbul: (37417, 480, 3)


In [29]:

window_index = 0  

print(f"Erstes Sliding Window für {location}:")
print(X_location[window_index])

print(f"\nErster Zeitschritt im Sliding Window für {location}:")
print(X_location[window_index, 0])  

print(f"\nZielwerte (y) für das erste Sliding Window für {location}:")
print(y_location[window_index])

Erstes Sliding Window für istanbul:
[[ 0.          0.4914611   0.        ]
 [ 0.          0.48766603  0.        ]
 [ 0.          0.48766603  0.        ]
 ...
 [ 0.          0.5256167  -1.        ]
 [ 0.          0.5256167  -1.        ]
 [ 0.          0.5256167  -1.        ]]

Erster Zeitschritt im Sliding Window für istanbul:
[0.        0.4914611 0.       ]

Zielwerte (y) für das erste Sliding Window für istanbul:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.03295025 0.03295025
 0.03295025 0.03295025 0.06689898 0.06689898 0.         0.13279947
 0.06689898 0.06689898 0.03295025 0.19969846 0.03295025 0.19969846
 0.13279947 0.09984923 0.13279947 0.16674821 0.06689898 0.13279947
 0.06689898 0.13279947 0.16674821 0.16674821 0.         0.06689898
 0.19969846 0.29954768 0.         0.29954768 0.33249793 0.19969846
 0.33249793 0.2326487  0.2326487  0.19969846 0.16674821 0.03295025
 0.09984923 0.         0.13279947 0.16674821 0.1327994

In [ ]:
def hyperparameter_tuning(trial):
    lstm_units_1 = trial.suggest_int('lstm_units_1', 16, 64, step=4)
    lstm_units_2 = trial.suggest_int('lstm_units_2', 16, 128, step=8)
    lstm_units_3 = trial.suggest_int('lstm_units_3', 32, 256, step=8)
    dropout_1 = trial.suggest_float('dropout_1', 0.1, 0.5)
    dropout_2 = trial.suggest_float('dropout_2', 0.1, 0.5)
    dropout_3 = trial.suggest_float('dropout_3', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 128, step=16)


    X_temp, X_test, y_temp, y_test, loc_temp, loc_test = train_test_split(
    X_final, y_final, location_ids,
    test_size=0.15,
    stratify=location_ids,
    random_state=42)

    gkf = GroupKFold(n_splits=5)
    
    val_losses = []

    for fold, (train_idx, val_idx) in enumerate(gkf.split(X_temp, y_temp, groups=loc_temp)):
        X_train, X_val = X_temp[train_idx], X_temp[val_idx]
        y_train, y_val = y_temp[train_idx], y_temp[val_idx]
            
        model = Sequential([
            Masking(mask_value=-1, input_shape=(X_train.shape[1], X_train.shape[2])),
            LSTM(lstm_units_1, activation="tanh", return_sequences=True),
            Dropout(dropout_1),
            LSTM(lstm_units_2, activation="tanh", return_sequences=True),
            Dropout(dropout_2),
            LSTM(lstm_units_3, activation="tanh"),
            Dropout(dropout_3), 
            Dense(96, activation='sigmoid')  # 96 Zeitschritte als Vorhersage (24h PV-Leistung)
        ])

        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss="mse", metrics=['mae'])

        history = model.fit(
            X_train, y_train,
            epochs=30,  
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            verbose=1  
        )



        val_loss = min(history.history['val_loss'])
        val_losses.append(val_loss)
    return np.mean(val_losses)

study = optuna.create_study(direction="minimize")
study.optimize(hyperparameter_tuning, n_trials=20)

print("Beste Hyperparameter:", study.best_params)




[I 2025-03-25 19:36:01,465] A new study created in memory with name: no-name-1fb9a94b-c4b0-4930-bf7b-e1589eb9d73f
C:\Users\opper\AppData\Local\Temp\ipykernel_4268\4232760410.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


In [1]:
best_params = study.best_params

final_model = Sequential([
    Masking(mask_value=-1, input_shape=(X_temp.shape[1], X_temp.shape[2])),
    LSTM(best_params['lstm_units_1'], activation="tanh", return_sequences=True),
    Dropout(best_params['dropout_1']),
    LSTM(best_params['lstm_units_2'], activation="tanh", return_sequences=True),
    Dropout(best_params['dropout_2']),
    LSTM(best_params['lstm_units_3'], activation="tanh"),
    Dropout(best_params['dropout_3']),
    Dense(96, activation='sigmoid')  
])

final_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss="mse", metrics=['mae'])

history= final_model.fit(
    X_temp, y_temp, 
    epochs=100, batch_size=best_params['batch_size'], 
    verbose=1
    )


NameError: name 'study' is not defined

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Trainingsverlust (MSE)')
plt.plot(history.history['mae'], label='Trainings-MAE')
plt.xlabel("Epochen")
plt.ylabel("Fehler")
plt.title("Trainingsverlauf des finalen Modells")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_mae = final_model.evaluate(X_test, y_test)
test_rmse = np.sqrt(test_loss)

print(f"\n Finaler Testverlust (MSE): {test_loss:.4f}")
print(f"\n Finaler Testverlust (RMSE): {test_rmse:.4f}")
print(f"Finaler Test-MAE: {test_mae:.4f}")

In [ ]:
print("\n Fehlerauswertung pro Standort:")
y_pred = final_model.predict(X_test)
unique_locations = np.unique(loc_test)

for loc in unique_locations:
    indices = np.where(loc_test == loc)[0]
    y_true_loc = y_test[indices]
    y_pred_loc = y_pred[indices]

    mse = mean_squared_error(y_true_loc, y_pred_loc)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_loc, y_pred_loc)

    print(f"  Standort {loc}:")
    print(f"    MSE : {mse:.4f}")
    print(f"    RMSE: {rmse:.4f}")
    print(f"    MAE : {mae:.4f}")

In [ ]:
y_pred = final_model.predict(X_test)

plt.figure(figsize=(10, 4))
plt.plot(y_test[0], label='Echte Werte')
plt.plot(y_pred[0], label='Vorhersage')
plt.title("Beispiel: Vorhersage vs. Realität (1 Testbeispiel)")
plt.xlabel("Zeitpunkte")
plt.ylabel("PV-Leistung (skaliert)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()